#### 2023/11/10 杉井さんの解析対象について理解し、データを作成する

In [26]:
import cv2
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
from preprocess_data_plusinfo import PreprocessDataPlusInfo

df_info = pd.read_excel('../data/20231111_all_drawing_info_maxpoints.xlsx')
df_info

,Unnamed: 0.1,Unnamed: 0,drawing_id,url,year,month,day,motif,times,name,rotate_type,img_url,max_points_max
0,0,0,0,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,和井田瞳,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,366
1,1,1,1,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,長田美穂子,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,643
2,2,2,2,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,佐藤宏哉,0,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,228
3,3,3,3,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,井原亜利紗,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,355
4,4,4,4,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,山田美沙紀,0,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,605
...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,954,954,954,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,2021,7,19,紙袋,3,安部琴海,90,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,89
955,955,955,955,http://skill.shinshu-u.ac.jp/w3kbko/5nur0x/str...,2016,5,2,紙箱,1,saito,0,http://skill.shinshu-u.ac.jp/w3kbko/5nur0x/str...,183
956,956,956,956,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,2022,6,13,紙箱,2,saito,0,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,145
957,957,957,957,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,2022,6,13,紙箱,3,saito,0,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,225


In [41]:
# 最大座標数が943を下回るドローイング
df_info_under943 = df_info[df_info['max_points_max'] < 943]

# 杉井さんが解析対象としていた2020年度以前のドローイング 合計806個のデータ
# そこからsaito先生の4ドローイングを除き、
df_info_under2021 = df_info_under943[df_info_under943['year'] < 2021]

df_info_sugii = pd.read_csv('../data/user_under_948_w_dir.tsv', delimiter='\t')
df_info

,Unnamed: 0.1,Unnamed: 0,drawing_id,url,year,month,day,motif,times,name,rotate_type,img_url,max_points_max
0,0,0,0,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,和井田瞳,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,366
1,1,1,1,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,長田美穂子,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,643
2,2,2,2,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,佐藤宏哉,0,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,228
3,3,3,3,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,井原亜利紗,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,355
4,4,4,4,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,山田美沙紀,0,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,605
...,...,...,...,...,...,...,...,...,...,...,...,...,...
954,954,954,954,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,2021,7,19,紙袋,3,安部琴海,90,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,89
955,955,955,955,http://skill.shinshu-u.ac.jp/w3kbko/5nur0x/str...,2016,5,2,紙箱,1,saito,0,http://skill.shinshu-u.ac.jp/w3kbko/5nur0x/str...,183
956,956,956,956,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,2022,6,13,紙箱,2,saito,0,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,145
957,957,957,957,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,2022,6,13,紙箱,3,saito,0,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,225


In [64]:
# 杉井さんの表にあって、浅井の表にないデータは0
# ⇒浅井の表で、杉井さんの解析対象ドローイングをすべて網羅できている
# sugii_url_notin = []        
# for sugii_url in df_info_sugii['URL']:
#     if sugii_url not in df_info['url'].to_list():
#         sugii_url_notin.append(sugii_url)

# 浅井の表にあって、杉井さんの表にないデータは159個
# ⇒浅井のデータ数959個 - 杉井さんの解析対象データ数800個 = 159個
asai_url_notin = []
for asai_url in df_info['url']:
    if asai_url not in df_info_sugii['URL'].to_list():
        asai_url_notin.append(asai_url)

drawing_id_notinsugii = [ drawing_id for drawing_id, url in zip(df_info['drawing_id'], df_info['url']) if url in asai_url_notin]
df_notin_sugii = df_info[df_info['drawing_id'].isin(drawing_id_notinsugii)]

df_notin_sugii.to_excel('../temp/20231111_indo_notinsugii.xlsx')

In [66]:
df_info_re = pd.read_excel('../temp/20231111_indo_notinsugii.xlsx')
df_info_re

,Unnamed: 0.2,Unnamed: 0.1,Unnamed: 0,drawing_id,url,year,month,day,motif,times,name,rotate_type,img_url,max_points_max
0,15,15,15,15,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,岩城知里,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,1455
1,16,16,16,16,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,峯本遥,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,964
2,29,29,29,29,http://skill.shinshu-u.ac.jp/2013/5-13/2013051...,2013,5,13,紙箱,2,長田美穂子,0,http://skill.shinshu-u.ac.jp/2013/5-13/2013051...,1652
3,30,30,30,30,http://skill.shinshu-u.ac.jp/2013/5-13/2013051...,2013,5,13,紙箱,2,小野茜,0,http://skill.shinshu-u.ac.jp/2013/5-13/2013051...,1029
4,33,33,33,33,http://skill.shinshu-u.ac.jp/2013/5-13/2013051...,2013,5,13,紙箱,2,髙橋なな,0,http://skill.shinshu-u.ac.jp/2013/5-13/2013051...,1311
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,954,954,954,954,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,2021,7,19,紙袋,3,安部琴海,90,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,89
155,955,955,955,955,http://skill.shinshu-u.ac.jp/w3kbko/5nur0x/str...,2016,5,2,紙箱,1,saito,0,http://skill.shinshu-u.ac.jp/w3kbko/5nur0x/str...,183
156,956,956,956,956,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,2022,6,13,紙箱,2,saito,0,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,145
157,957,957,957,957,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,2022,6,13,紙箱,3,saito,0,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,225


In [ ]:
# ビューワ上にある各ドローイング画像のURLから取ってきた画像を保存
# folder_name_viewer_notrotate = '20231110_drawingimg_viewer_notrotate'

# for img_url, drawing_id in zip(df_info['img_url'], df_info['drawing_id']):
#     response = requests.get(img_url, auth=('19t2003a', 'ireneRED77'))
#     content = response.content
#     file_name = f'../temp/{folder_name_viewer_notrotate}/drawing_id_{drawing_id}.jpg'
#     print(file_name)
#     with open(file_name, 'wb') as f:
#         f.write(content)

In [31]:
# 浅井のプログラムで作成したグラフを保存
# folder_name_plot_rotate = '../temp/20231110_drawingimg_plot_rotate'

# drawing_id_start = 0
# drawing_id_end = 165
# df_info_year = df_info[drawing_id_start:drawing_id_end]

# for url, rotate_type, drawing_id in zip(df_info_year['url'], df_info_year['rotate_type'], df_info_year['drawing_id']):
#     data = PreprocessDataPlusInfo(url, rotate_type)
#     file_name = f'../temp/{folder_name_plot_rotate}/drawing_id_{drawing_id}.jpg'
#     print(file_name)
#     data.draw_allstroke(file_name)

In [4]:
import io
columns = [ 'time', 'alpha', 'max_points', 'ptx_anoto', 'pty_anoto', 'shape_str',
                    'pressure_style', 'len_mm_acc', 'len_anoto', 'pressure_avg', 'shape_int']

# URLからデータフレームを取得 & ストローク長の列を追加
def get_df(url):
    content = requests.get(url, auth=('19t2003a', 'ireneRED77')).content
    df = pd.read_table(io.StringIO(content.decode('utf-8')), header = None)
    df.columns = columns
    st_len_mm = [l*0.3 for l in df['len_anoto']]
    df['len_mm'] = st_len_mm
    # ストローク固有のidを付与
    df_reindex = df.reset_index()
    df_reindex = df_reindex.rename(columns={'index': 'stroke_id'})
 
    return df_reindex

#################################################
num = 15
data = PreprocessDataPlusInfo(df_info['url'][num], df_info['rotate_type'][num])
file_name_plot = '../temp/test.jpg'
data.draw_allstroke(file_name_plot)


num = 15
df_test = get_df(df_info['url'][num])

#### 最大座標数が943以上のドローイングがあるっぽい

In [15]:
max_points_list = []
# df_info_range = df_info[854:]

for url, drawing_id in zip(df_info['url'], df_info['drawing_id']):
    if drawing_id == 698 or drawing_id == 853:
       max_points_list.append(-1)
       print('-----', drawing_id)
    else:
        df = get_df(url)
        max_points_max = df['max_points'].max()
        max_points_list.append(max_points_max)
        print(drawing_id)

# df_max_points = pd.DataFrame({'url': df_info['url'],
#                               'drawing_id': df_info['drawing_id'],
#                               'max_points': max_points_list})

df_info['max_points_max'] = max_points_list

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [25]:
df_info.to_excel('../temp/20231111_all_drawing_info_maxpoints.xlsx')

In [70]:
df = pd.read_excel('../data/20231111_all_drawing_info_maxpoints.xlsx', index_col=0)

In [72]:
del df['Unnamed: 0']

,drawing_id,url,year,month,day,motif,times,name,rotate_type,img_url,max_points_max
0,0,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,和井田瞳,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,366
1,1,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,長田美穂子,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,643
2,2,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,佐藤宏哉,0,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,228
3,3,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,井原亜利紗,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,355
4,4,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,山田美沙紀,0,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,605
...,...,...,...,...,...,...,...,...,...,...,...
954,954,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,2021,7,19,紙袋,3,安部琴海,90,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,89
955,955,http://skill.shinshu-u.ac.jp/w3kbko/5nur0x/str...,2016,5,2,紙箱,1,saito,0,http://skill.shinshu-u.ac.jp/w3kbko/5nur0x/str...,183
956,956,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,2022,6,13,紙箱,2,saito,0,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,145
957,957,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,2022,6,13,紙箱,3,saito,0,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,225


In [79]:
all_one = [1 for i in range(len(df))]

In [81]:
isinsugii_id = []
for i, drawing_id in enumerate(df['drawing_id']):
    if drawing_id in drawing_id_notinsugii:
        isinsugii_id.append(0)
    else:
        isinsugii_id.append(1)

df['isin_sugiisan'] = isinsugii_id

In [84]:
df.to_excel('../temp/20231111_all_drawing_info.xlsx')

In [87]:
df_info = pd.read_excel('../temp/20231111_all_drawing_info.xlsx', index_col=0)

In [88]:
df_info

,drawing_id,url,year,month,day,motif,times,name,rotate_type,img_url,max_points_max,isin_sugiisan
0,0,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,和井田瞳,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,366,1
1,1,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,長田美穂子,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,643,1
2,2,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,佐藤宏哉,0,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,228,1
3,3,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,井原亜利紗,180,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,355,1
4,4,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,2013,4,18,紙箱,1,山田美沙紀,0,http://skill.shinshu-u.ac.jp/2013/4-18/2013041...,605,1
...,...,...,...,...,...,...,...,...,...,...,...,...
954,954,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,2021,7,19,紙袋,3,安部琴海,90,http://skill.shinshu-u.ac.jp/2021/7-19/2021071...,89,0
955,955,http://skill.shinshu-u.ac.jp/w3kbko/5nur0x/str...,2016,5,2,紙箱,1,saito,0,http://skill.shinshu-u.ac.jp/w3kbko/5nur0x/str...,183,0
956,956,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,2022,6,13,紙箱,2,saito,0,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,145,0
957,957,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,2022,6,13,紙箱,3,saito,0,http://skill.shinshu-u.ac.jp/i0zjtz/paper_bag_...,225,0
